# MNIST with CNN

In [ ]:
### switch off deprecation and future warnings
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    return [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']

# Test whether the GPU has been setup correctly, if correct, ['/device:GPU:0'] will be printed
get_available_gpus()

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

### Helper Functions

#### INIT WGTS

#### INIT BIAS

#### CONV2D

Function to help intialize random weights for fully connected or convolutional layers, we leave the shape attribute as a parameter for this.

In [ ]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

Same as init_weights, but for the biases

In [ ]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

Create a **2D convolution** using builtin conv2d from TF. From those docs:

Computes a 2-D convolution given 4-D `input` and `filter` tensors.

Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
and a filter / kernel tensor of shape
`[filter_height, filter_width, in_channels, out_channels]`, this op
performs the following:

1. Flattens the filter to a 2-D matrix with shape
   `[filter_height * filter_width * in_channels, output_channels]`.
2. Extracts image patches from the input tensor to form a *virtual*
   tensor of shape `[batch, out_height, out_width,
   filter_height * filter_width * in_channels]`.
3. For each patch, right-multiplies the filter matrix and the image patch
   vector.


In [ ]:
def conv2d(x, W):
    # x --> [batch, H, W, Channels]
    # W --> [filter H, filter W, Channels IN, Channel OUT]
    # Using convenience function conv2d pass in the tensors, stride one 
    # in all directions
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

Create a **max pooling layer**, again using built in TF functions:

Performs the max pooling on the input.

    Args:
      value: A 4-D `Tensor` with shape `[batch of images, height, width, channels]` and
        type `tf.float32`.
      ksize: A list of ints that has length >= 4.  The size of the window for
        each dimension of the input tensor.
      strides: A list of ints that has length >= 4.  The stride of the sliding
        window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. 

In [ ]:
def max_pool_2by2(x):
    # x --> [batch, h, w, channels], a 4D Tensor
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], # we do pooling along the height and width
                          strides=[1, 2, 2, 1], # Same for strides
                          padding='SAME')

- So again we're only really concerned as far as the pooling along in individual images height and width
which is why we're getting these images of 1 2 2 1 and 1 2 2 1.

So now we have our convenience functions that basically already set these parameters for us that we
don't need to provide them every time.


**Using the conv2d function**, we'll return an actual convolutional layer here that uses an ReLu activation.



In [ ]:
"""
Now we're going to create two functions where we actually
create the layers.
So the first one is going to be the convolutional air.
So this function in fact we'll just call it convolutional layer
is going to 
take some input X and then the shape parameter so we'll create
the weights by asking in weights with the shape and the biases 
are just going to run along the third dimension. So I'll say it
biases and then we're just going to pass 
in shape three here.I'm going to use a rectified linear unit as 
the activation function pass in the to the convolution of input X 
with those weights Plus biases
"""
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

This is a normal fully connected layer

**Narrative**:
So we'll make this function say normal full layer. And this is going to take in some input layer of a size and we'll say the input size is equal to the integer of the input layer. And we want to get the shape of the index one they mention will say w initialize the weights and then we're going to pass pass in input size by size to create that layer and then we'll say B = 

In [ ]:
def normal_full_layer(input_layer, size):  #FCN
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

So now we have two functions one creates a convolutional air one creates a normal fully connected layer.
So it's time to actually build out multiple layers along with the placeholders do a loss function do
an optimizer initialize the variables and run the session.



**What is the hardest part?**
I would say the hardest part of all of this is keeping in mind your dimensions.
Once you do this multiple times if image sets you start to get pretty.
So this idea of these for them actions just across the board you have your batch your height your with
your channels but in the very beginning it can be hard to keep track or just try to visualize these
kind of for the tensors floating around in the layers so it definitely takes a lot of practice as a
quick note to kind of understand these dimensions and have them be intuitive.
It's definitely not intuitive at first.

## Next

now it's go ahead and start building out
our **CNN**


### Placeholders


So the first thing we're going to do here is create our placeholders.
So we have X is going to be a placeholder.

his is our actual data so I'll say to 32 and we'll see the shape we'll say none because that's the
size of that batch.

And then 784 because that's how many pixels are in the data. So that's 28 times 28.

Then we have y_true. These are the y true labels remember it's one encoded. Which means when I say to flip 32 I'll pass in the shape as none because it's again going to be the batch size and then 10 because it's 0 through 9 because of 1-hot encoded.

So those are my placeholders.


In [ ]:
"""
So we have X is going to be a placeholder.
This is our actual data so I'll say to 32 and we'll see 
the shape we'll say none because that's the
size of that batch.

And then 784 because that's how many pixels are in the data.
So that's 28 times 28.

Then we have y_true. These are the y true labels remember
it's one hot encoded. 
Which means when I say to flip 32 I'll pass in the shape as
none because it's again 
going to be the batch size and then 10 because 
it's 0 through 9 because of 1-hot encoded.
So those are my placeholders.
"""
x = tf.placeholder(tf.float32,shape=[None,784])

In [ ]:
y_true = tf.placeholder(tf.float32,shape=[None,10])
# I only use those for the feed dictionaries and 
#now it's time to actually create the layers
# so the first one we're going to say is the image layer.

### Layers

So why am I doing this because I want to reshape this flattened out array of 784 into the image again.
So 28 by 28. That's the height and width.

**Tip**: Do you know why we do -1

Answer: It means, that the size of the dimension, for which you passed -1, is being inferred. Thus,
A.reshape(-1, 28*28) means, "reshape A so that its second dimension has a size of 28*28 and calculate the correct size of the first dimension".

In [ ]:
#Reshape
# So why am I doing this because I want to reshape 
# this flattened out array into the image again.
# So 28 by 28 That's the height and width.
# I remember this one. It's just greyscale. 
# There's only one color channel there. So now that's my image.

x_image = tf.reshape(x,[-1,28,28,1])

In [ ]:
# Using a 6by6 filter here, used 5by5 in video,
# you can play around with the filter size
# You can change the 32 output, that essentially 
#represents the amount of filters used
# You need to pass in 32 to the next input though, 
#the 1 comes from the original input of 
# a single image.
convo_1 = convolutional_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [ ]:
# Using a 6by6 filter here, used 5by5 in video, 
# you can play around with the filter size
# You can actually change the 64 output if you want,
# you can think of that as a representation
# of the amount of 6by6 filters used.
convo_2 = convolutional_layer(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [ ]:
# Why 7 by 7 image? Because we did 2 pooling layers, so (28/2)/2 = 7
# 64 then just comes from the output of the previous Convolution
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [ ]:
# NOTE THE PLACEHOLDER HERE!
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [ ]:
y_pred = normal_full_layer(full_one_dropout,10)

### Loss Function

So let's start for a last function we'll use cross entropy to be our loss function and we essentially just use the built in functions that are already TF for us so we say reduce mean to take an average and we'll say soft Max cross entropy and we're going to say that the labels is equal to y true which will be provided by our dictionary later on and then we'll say logits=y_pred and so on

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))

### Optimizer

So our optimizer, we're going to go ahead and use an adam optimizer say Adam optimizer and then we'll say the learning rates is 0.001 and then our trainer is just going to be for this optimizer to minimize our loss which in our case is the cross entropy loss.

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

### Intialize Variables



In [ ]:
init = tf.global_variables_initializer()

### Session

`sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})`

So remember that we actually have three placeholders. We have the two classic ones which is just x and y. So we'll say X is batch x and y true is that y. And then we have the hold probability because remember we are designing that dropout layer appear. We have this whole probability which is a placeholder and that basically is the probability that a neuron is held during dropout. So during training we'll go ahead and say 50 percent. So each near on has a 50 percent chance of being held essentially randomly dropping out half our neurons.

```python
if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')
```
Every 100th step we are going to do something:

- Report back our accuracy on the test set.
- We did the same for our MLP test (OK so we have our matches remember that's a list of booleans So we want to cast it into a list of floats and then take the average.)

In [ ]:
steps = 5000

# Creates a session with log_device_placement set to True.
config = tf.ConfigProto(log_device_placement = True)
config.gpu_options.per_process_gpu_memory_fraction = 1.0

session = tf.Session( config = config )

with session as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        # Grabbing from the next batch
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        # note: we have 3 placeholders, remmb
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,
                                          y_true:mnist.test.labels,
                                          hold_prob:1.0}))
            print('\n')

## Sweet stuff!